In [ ]:
import sys
import localSettings as ls
import os
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
from scipy.optimize import curve_fit

def gauss(x,mu,sigma,A):
    norm = A/(np.sqrt(2*np.pi)*sigma)
    exp  = np.exp(-((x-mu)**2)/(2*sigma*sigma))
    return norm * exp

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
#if USEBDT:
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward
import pandas as pd

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
import load_data_run123

In [ ]:
PATHRUN1 = '/Users/dcaratelli/data/searchingfornues/v08_00_00_48/farsidebands/nslice/'
PATHRUN2 = '/Users/dcaratelli/data/searchingfornues/v08_00_00_48/farsidebands/nslice/'
PATHRUN3 = '/Users/dcaratelli/data/searchingfornues/v08_00_00_48/farsidebands/nslice/'
PATHRUN4 = '/Users/dcaratelli/data/searchingfornues/v08_00_00_63/run4/'

In [ ]:
FILERUN1 = 'run1_neutrinoselection_filt_numu_ALL.root'
FILERUN2 = 'run2_neutrinoselection_filt_numu_ALL.root'
FILERUN3 = 'run3_neutrinoselection_filt_numu_ALL.root'
FILERUN4 = 'bnb_on_run4a_reco2_v08_00_00_63_POT_DEBUG_run4a_reco2_beam_good.root'

In [ ]:
fold = 'nuselection'
tree = 'NeutrinoSelectionFilter'

In [ ]:
VARDICT = load_data_run123.get_variables()
variables = VARDICT['VARIABLES']

In [ ]:
print ("topological_score" in variables)

In [ ]:
up1 = uproot.open(PATHRUN1+FILERUN1)[fold][tree]
df1 = up1.pandas.df(variables, flatten=False)

In [ ]:
up2 = uproot.open(PATHRUN2+FILERUN2)[fold][tree]
df2 = up2.pandas.df(variables, flatten=False)

In [ ]:
up3 = uproot.open(PATHRUN3+FILERUN3)[fold][tree]
df3 = up3.pandas.df(variables, flatten=False)

In [ ]:
up4 = uproot.open(PATHRUN4+FILERUN4)[fold][tree]
df4 = up4.pandas.df(variables, flatten=False)

In [ ]:
load_data_run123.process_uproot_numu(up1,df1)
load_data_run123.process_uproot_numu(up2,df2)
load_data_run123.process_uproot_numu(up3,df3)
load_data_run123.process_uproot_numu(up4,df4)

In [ ]:
import unblinding_far_sideband

In [ ]:
df1['bnbdata']  = np.ones_like(df1['nslice'])
df2['bnbdata']  = np.ones_like(df2['nslice'])
df3['bnbdata']  = np.ones_like(df3['nslice'])
df4['bnbdata']  = np.ones_like(df4['nslice'])

df1['extdata']  = np.zeros_like(df1['nslice'])
df2['extdata']  = np.zeros_like(df2['nslice'])
df3['extdata']  = np.zeros_like(df3['nslice'])
df4['extdata']  = np.zeros_like(df4['nslice'])
#df['extdata']  = np.zeros_like(df['nslice'])

In [ ]:
QUERY = unblinding_far_sideband.NUMUPRESEL# + ' and n_muons_tot > 0'
QUERY = unblinding_far_sideband.NUMUSEL

In [ ]:
#QUERY = ' topological_score > 0.5 and nslice == 1'

In [ ]:
print (QUERY)

In [ ]:
QUERY += ' and (run < 19080 or run > 19420)'

In [ ]:
dfsel1 = df1.query(QUERY)
dfsel2 = df2.query(QUERY)
dfsel3 = df3.query(QUERY)
dfsel4 = df4.query(QUERY)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig = plt.figure(figsize=(6,6))

BINS = np.linspace(0.15,1.55,14)
VAR = 'neutrino_energy'

plt.hist(dfsel1[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 1')
plt.hist(dfsel2[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 2')
plt.hist(dfsel3[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 3')
plt.hist(dfsel4[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 4')

plt.xlabel('neutrino energy [GeV]')
plt.legend(fontsize=14)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))

BINS = np.linspace(-1,1,20)
VAR = 'muon_theta'

plt.hist(dfsel1[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 1')
plt.hist(dfsel2[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 2')
plt.hist(dfsel3[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 3')
plt.hist(dfsel4[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 4')

plt.xlabel('muon cos-theta')
plt.legend(fontsize=14,loc=2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))

BINS = np.linspace(0.15,1.55,14)
VAR = 'muon_energy'

plt.hist(dfsel1[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 1')
plt.hist(dfsel2[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 2')
plt.hist(dfsel3[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 3')
plt.hist(dfsel4[VAR].values,bins=BINS,histtype='step',lw=2,density=True,label='Run 4')

plt.xlabel('muon energy [GeV]')
plt.legend(fontsize=14,loc=1)
plt.show()